In [3]:
if (!require("tidyverse")) { install.packages("tidyverse")
  require("tidyverse") };
  library("tidyverse");

if (!require("reshape")) { install.packages("reshape")
  require("reshape") };
library("reshape");

if (!require("data.table")) { install.packages("data.table")
  require("data.table") };
library("data.table");

Loading required package: tidyverse
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.0       v purrr   0.2.5  
v tibble  2.0.1       v dplyr   0.8.0.1
v tidyr   0.8.1       v stringr 1.3.1  
v readr   1.1.1       v forcats 0.3.0  
Warning message:
"package 'ggplot2' was built under R version 3.5.2"Warning message:
"package 'tibble' was built under R version 3.5.2"Warning message:
"package 'dplyr' was built under R version 3.5.2"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Loading required package: reshape

Attaching package: 'reshape'

The following object is masked from 'package:dplyr':

    rename

The following objects are masked from 'package:tidyr':

    expand, smiths

Loading required package: data.table

Attaching package: 'data.table'

The following object is masked from 'package:reshape':

    melt

The following object

In [4]:
MyData <- read.csv(file="all_stocks_2017-2018.csv", header=TRUE, sep=",");
tail(MyData,5)

,Date,Open,High,Low,Close,Volume,Name
7777,12/22/2017,71.42,71.87,71.22,71.58,10979165,AABA
7778,12/26/2017,70.94,71.39,69.63,69.86,8542802,AABA
7779,12/27/2017,69.77,70.49,69.69,70.06,6345124,AABA
7780,12/28/2017,70.12,70.32,69.51,69.82,7556877,AABA
7781,12/29/2017,69.79,70.13,69.43,69.85,6613070,AABA


In [5]:
NewData <- MyData %>%
  filter(str_detect(Name, "AAPL|MSFT|AMZN|GOOGL"))

In [6]:
PivotedData <- cast(NewData, Date ~ Name, value = "Close")
PivotedData <- PivotedData[order(as.Date(PivotedData$Date, format="%m/%d/%Y")), ]
tail(PivotedData)

,Date,AAPL,AMZN,GOOGL,MSFT
73,12/21/2017,175.01,1174.76,1070.85,85.50
74,12/22/2017,175.01,1168.36,1068.86,85.51
75,12/26/2017,170.57,1176.76,1065.85,85.40
76,12/27/2017,170.60,1182.26,1060.20,85.71
77,12/28/2017,171.08,1186.10,1055.95,85.72
78,12/29/2017,169.23,1169.47,1053.40,85.54


In [7]:
# Turn data into index (rownames in R) to simplify transformations later on
rownames(PivotedData) <- PivotedData$Date
PivotedData$Date <- NULL

## Understanding of vectorized functions
- Lead vs lag in R, while just + or - in Python. -> Python is 
- lapply vs 

In [8]:
Shifted <- lapply(PivotedData,lag,n=7)
head(data.frame(Shifted),10)

AAPL,AMZN,GOOGL,MSFT
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
116.15,753.67,808.01,62.58
116.02,757.18,807.77,62.30
116.61,780.45,813.02,62.30


In [9]:
Rate_of_change <-  PivotedData/Shifted - 1
head(data.frame(Shifted),10)
head(Rate_of_change,10)

AAPL,AMZN,GOOGL,MSFT
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
NA,NA,NA,NA
116.15,753.67,808.01,62.58
116.02,757.18,807.77,62.30
116.61,780.45,813.02,62.30


,AAPL,AMZN,GOOGL,MSFT
1/3/2017,NA,NA,NA,NA
1/4/2017,NA,NA,NA,NA
1/5/2017,NA,NA,NA,NA
1/6/2017,NA,NA,NA,NA
1/9/2017,NA,NA,NA,NA
1/10/2017,NA,NA,NA,NA
1/11/2017,NA,NA,NA,NA
1/12/2017,0.02668963,0.07957063,0.02663333,0.0004793864
1/13/2017,0.02602999,0.07918857,0.02868391,0.0064205457
1/17/2017,0.02907126,0.03750400,0.01776094,0.0036918138


In [10]:
Rate_of_change$Date <- rownames(Rate_of_change)

MeltedData <- melt(Rate_of_change, id="Date")
colnames(MeltedData)[3] <- "delta_7"
#write.csv(Rate_of_change, file = "Rate_of_change.csv")
head(MeltedData,20)

Date,variable,delta_7
1/3/2017,AAPL,NA
1/4/2017,AAPL,NA
1/5/2017,AAPL,NA
1/6/2017,AAPL,NA
1/9/2017,AAPL,NA
1/10/2017,AAPL,NA
1/11/2017,AAPL,NA
1/12/2017,AAPL,0.026689625
1/13/2017,AAPL,0.026029995
1/17/2017,AAPL,0.029071263


In [11]:
# We want to shift the data to create a dataset that contains records moving backwards by day 7 days (1 week), 14 days (2 weeks), 21 days, and 28 days.

delta_dict <- list(7,14,21,28)
names(delta_dict) <- list("delta_7","delta_14","delta_21","delta_28")

In [12]:
melted_dfs <- list()
for (i in delta_dict)
{
  transit <- lapply(PivotedData,lag,n=i)
  rate_change <-  PivotedData/transit - 1
  rate_change$Date <- rownames(rate_change)
  melted_dfs[[paste("delta_",i)]] <- melt(rate_change, id="Date")
  colnames(melted_dfs[[paste("delta_",i)]])[2] <- "Name"
  colnames(melted_dfs[[paste("delta_",i)]])[3] <- paste("delta_",i)
}

In [13]:
# We need a seperate section to shift 7 days forward (to act as a future predictor)
transit <- lapply(PivotedData,lead,n=7)
rate_change <-  PivotedData/transit - 1
rate_change$Date <- rownames(rate_change)
melted_dfs[["return_7"]] <- melt(rate_change, id="Date")
colnames(melted_dfs[["return_7"]])[2] <- "Name"
colnames(melted_dfs[["return_7"]])[3] <- "return_7"

In [14]:
Merged_Return <- Reduce(function(x,y) merge(x,y,by=c("Date","Name"),all=TRUE),melted_dfs)
base_Data <- select(NewData,'Date', 'Name', 'Volume', 'Close')

Merged_Data <- merge(base_Data,Merged_Return,by=c("Date","Name"),all=TRUE)
Merged_Data <- Merged_Data[order(as.Date(Merged_Data$Date, format="%m/%d/%Y")), ] 
# Merged_Data <- Merged_Data[order(Merged_Data$Name)]

In [15]:
Merged_Data

,Date,Name,Volume,Close,delta_ 7,delta_ 14,delta_ 21,delta_ 28,return_7
53,1/3/2017,AAPL,28781865,116.15,NA,NA,NA,NA,-0.0259958071
54,1/3/2017,AMZN,3521066,753.67,NA,NA,NA,NA,-0.0737058158
55,1/3/2017,GOOGL,1959033,808.01,NA,NA,NA,NA,-0.0259424011
56,1/3/2017,MSFT,20694101,62.58,NA,NA,NA,NA,-0.0004791567
65,1/4/2017,AAPL,21118116,116.02,NA,NA,NA,NA,-0.0253696237
66,1/4/2017,AMZN,2510526,757.18,NA,NA,NA,NA,-0.0733778789
67,1/4/2017,GOOGL,1515339,807.77,NA,NA,NA,NA,-0.0278840831
68,1/4/2017,MSFT,21339969,62.30,NA,NA,NA,NA,-0.0063795853
69,1/5/2017,AAPL,22193587,116.61,NA,NA,NA,NA,-0.0282500000
70,1/5/2017,AMZN,5830068,780.45,NA,NA,NA,NA,-0.0361482982


In [16]:
write.csv(Merged_Data, file = "MergedData.csv")